# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data_path = "../WeatherPy/cities_df.csv"
weather_data = pd.read_csv(weather_data_path)

# Drop unnamend column
weather_data = weather_data[['City', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind_Speed']]
weather_data.head()


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,Geraldton,-28.7667,114.6000,31.23,22,7,2.57
1,Pinar del Río,22.4175,-83.6981,19.58,83,26,3.14
2,Corte Madera,37.9255,-122.5275,15.74,74,0,4.02
3,Skeldon,5.8833,-57.1333,23.64,95,99,1.49
4,Huitán,15.1000,-91.6167,11.87,86,93,0.77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#----------------------------------------------------------------------------------------------------
# FIND CITIES WITH PERFECT CONDITIONS. DROP ANY ROWS THAT DON"T CONTAIN ALL THREE CONDITIONS BELOW
#--- TEMPERATURE BETWEEN 21 & 27 DEGREES CENTIGRADE
#--- WIND SPEED LESS THAN 10 MPH
#--- ZERO CLOUDINESS
#----------------------------------------------------------------------------------------------------

perfect_weather_df = weather_data[(weather_data['Temperature'] >= 21) & (weather_data['Temperature'] < 26) & (weather_data['Wind_Speed'] < 10) & (weather_data['Cloudiness'] == 0)]

# Visualize
print(len(perfect_weather_df))
perfect_weather_df.head()


16


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
137,Radcliff,37.8404,-85.9491,21.58,55,0,3.13
167,San Carlos Park,26.4673,-81.8015,22.98,74,0,5.14
226,Clearwater,27.9659,-82.8001,22.96,70,0,6.26
281,Cabot,34.9745,-92.0165,22.79,72,0,0.00
282,St. Marys,30.7305,-81.5465,22.01,79,0,4.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#---------------------------------------------
# STORE INTO VARIABLE NAMED 'HOTEL_DF'
# ADD A 'HOTEL NAME' COLUMN TO THE DATAFRAME
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ''
hotel_df.head()

#---------------------------------------------
# SET PARAMETERS TO SEARCH FOR HOTELS WITHIN 5000 METERS
# HIT THE GOOGLE PLACES API FOR EACH CITY"S COORDINATES
# STOR THE FIRST HOTEL RESULT INTO THE DATAFRAME
# PLOT MARKERS ON TOP OF THE HEATMAP
#---------------------------------------------



C:\Users\rjtur\AppData\Local\Temp/ipykernel_17108/2665564006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
137,Radcliff,37.8404,-85.9491,21.58,55,0,3.13,
167,San Carlos Park,26.4673,-81.8015,22.98,74,0,5.14,
226,Clearwater,27.9659,-82.8001,22.96,70,0,6.26,
281,Cabot,34.9745,-92.0165,22.79,72,0,0.00,
282,St. Marys,30.7305,-81.5465,22.01,79,0,4.12,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
